In [1]:
# Needed to reload modified modules
%load_ext autoreload
%autoreload 2

In [2]:
# Evaluate the ML model
import os
import time

import numpy as np
import matplotlib.pyplot as plt
from utils import sample_files, decoder_files_to_tensors
from utils import decoder_files_to_tensors
from models import Decoder
# from local_utils import get_best_model_timestamp

# data_dir = '/eos/user/k/kiliakis/tomo_data/datasets_decoder_02-12-22'
data_dir = './tomo_data/datasets_decoder_TF_24-03-23'
dataset_percent = 0.01
normalization = 'minmax'
ps_normalize = 'off'

var_names = ['phEr', 'enEr', 'bl',
             'inten', 'Vrf', 'mu', 'VrfSPS']

# timestamp = '2023_03_09_16-09-44'
timestamp = 'best_decoder_TF'



2023-07-03 18:58:41.128629: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 18:58:47.343783: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-03 18:58:48.219440: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kiliakis/install/anaconda3/lib:/home/kiliakis/install/lib
2023-07-03 18:58:48

In [3]:
# Initialize train/ test / validation paths
ML_dir = os.path.join(data_dir, 'ML_data')
TESTING_PATH = os.path.join(ML_dir, 'TESTING')
assert os.path.exists(TESTING_PATH)


# First the training data
file_names = sample_files(TESTING_PATH, dataset_percent)

start_t = time.time()
# read input, divide in features/ label, create tensors
x_test, y_test = decoder_files_to_tensors(
    file_names, normalization=normalization, ps_normalize=ps_normalize)
total_time = time.time() - start_t
print(
    f'Elapsed time: {total_time:.3f}, Per file: {total_time/len(file_names):.3f}')



2023-07-03 18:59:37.500417: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kiliakis/install/anaconda3/lib:/home/kiliakis/install/lib
2023-07-03 18:59:37.512774: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-03 18:59:37.512808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sy153): /proc/driver/nvidia/version does not exist
2023-07-03 18:59:37.549660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appr

Elapsed time: 47.778, Per file: 0.163


In [4]:
# Model to load
print('Decoder: ', timestamp)

# Initialize directories
trial_dir = os.path.join('./trials/', timestamp)
weights_dir = os.path.join(trial_dir, 'weights')
assert os.path.exists(weights_dir)

decoder = Decoder()
decoder.load(weights_dir)


Decoder:  best_decoder_TF


In [5]:
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_1 (Dense)             (None, 256)               2304      
                                                                 
 Dense_2 (Dense)             (None, 1024)              263168    
                                                                 
 Expand (Dense)              (None, 8192)              8396800   
                                                                 
 Reshape (Reshape)           (None, 16, 16, 32)        0         
                                                                 
 CNN_1 (Conv2DTranspose)     (None, 32, 32, 32)        82976     
                                                                 
 CNN_2 (Conv2DTranspose)     (None, 64, 64, 16)        41488     
                                                                 
 CNN_3 (Conv2DTranspose)     (None, 128, 128, 8)      

In [7]:
import visualkeras

visualkeras.layered_view(decoder.model, to_file=os.path.join(trial_dir, 'decoder-visual.png'), 
                         legend=True, index_ignore=[], max_z=100, max_xy=400, draw_volume=True).show()

In [ ]:
from models import Encoder


In [ ]:
# drop column from y_train, y_valid
# x_test_red = tf.concat([tf.expand_dims(tf.gather(x_test, i, axis=1), axis=1)
#                      for i in train_cfg['loss_weights']], -1)
# print('x_test shape: ', x_test_red.shape)

In [ ]:
# Evaluate the model on the test and validation data
test_loss = decoder.model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss:.4e}')

# get predictions
test_pred = decoder.model.predict(x_test, verbose=False)
y_test = np.array(y_test)

# Calculate error per variable
# mses = mean_squared_error(y_test, test_pred, multioutput='raw_values')

# valid_loss = decoder.evaluate(x_valid, y_valid)
# print(f'Valid loss: {valid_loss:.4e}')
# valid_pred = decoder.predict(x_valid, verbose=False)

In [ ]:
%matplotlib inline

print(np.max(y_test))
print(np.min(y_test))
mse_image = np.mean((y_test - test_pred) ** 2, axis=0)
mse_image = mse_image.reshape((128, 128))

me_image = np.mean(np.abs(y_test - test_pred), axis=0)
me_image = me_image.reshape((128, 128))

# Create 3x3 grid of figures
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 8))

ax.set_xticks([])
ax.set_yticks([])
# show the image
plt.imshow(me_image, cmap='jet', aspect='auto')
plt.colorbar()
# Set the label
# title = ','.join([f'{num:.1f}' for num in samples_X[i]])
ax.set_title(f'Mean Diff.')

# for i in range(len(axes)):
#     ax = axes[i]


In [ ]:
%matplotlib inline
# plot some of the outputs

nrows = 5
# Get nrows * nrows random images
sample = np.random.choice(np.arange(len(y_test)),
                          size=nrows, replace=False)

samples_real = y_test[sample]
samples_pred = test_pred[sample]

# Create 3x3 grid of figures
fig, axes = plt.subplots(ncols=3, nrows=nrows, figsize=(12, 20))
# axes = np.ravel(axes)
for i in range(nrows):
    ax = axes[i][0]
    plt.sca(ax)
    ax.set_xticks([])
    ax.set_yticks([])
    # show the image
    ax.imshow(samples_real[i], cmap='jet')
    # Set the label
    # title = ','.join([f'{num:.1f}' for num in samples_X[i]])
    ax.set_title(f'True')
    plt.tight_layout()

    ax = axes[i][1]
    plt.sca(ax)
    ax.set_xticks([])
    ax.set_yticks([])
    # show the image
    ax.imshow(samples_pred[i], cmap='jet')
    # Set the label
    # title = ','.join([f'{num:.1f}' for num in samples_X[i]])
    ax.set_title(f'Predicted')
    plt.tight_layout()

    ax = axes[i][2]
    plt.sca(ax)
    ax.set_xticks([])
    ax.set_yticks([])
    # show the image
    plt.imshow(np.abs(samples_real[i] -
              samples_pred[i]), cmap='jet', vmin=0, vmax=1,
              aspect='auto')
    plt.colorbar()
    # Set the label
    # title = ','.join([f'{num:.1f}' for num in samples_X[i]])
    ax.set_title(f'Diff')
    plt.tight_layout()
